In [4]:
import lightgbm
lightgbm.__version__

'3.3.2'

In [3]:
import xgboost
xgboost.__version__

'1.5.0'

In [2]:
import hyperopt
hyperopt.__version__

'0.2.7'

# HyperOpt를 이용한 하이퍼파라미터 튜닝

1. 검색공간 설정
2. 대체모델을 위한 목적함수 지정
3. 최적의 파라미터를 유추

In [ ]:
# params = {
#     'max_depth': [2,4,6,8,10],
#     'min_samples_split' : [2,5,10,20],
#     'min_samples_leaf': [1,2,4,8]
# }

In [5]:
from hyperopt  import hp

In [6]:
# 검색공간
search_space = { 'x': hp.quniform('x', -10, 10, 1), 
                'y': hp.quniform('y', -15, 15, 1)}


In [7]:
# 목적함수
def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    retval = x**2 - 20*y

    return retval

## 최적 입력값을 유추

In [8]:
from hyperopt import fmin, tpe, Trials
import numpy as np

trial_val = Trials()

best_01 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=5,
     trials=trial_val, rstate=np.random.default_rng(seed=0)
     )
best_01

100%|██████████| 5/5 [00:00<00:00, 1248.60trial/s, best loss: -224.0]


{'x': -4.0, 'y': 12.0}

In [9]:
best_02 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=20,
     trials=trial_val, rstate=np.random.default_rng(seed=0)
     )
best_02

100%|██████████| 20/20 [00:00<00:00, 832.60trial/s, best loss: -296.0]


{'x': 2.0, 'y': 15.0}

In [15]:
trial_val.best_trial

{'state': 2,
 'tid': 10,
 'spec': None,
 'result': {'loss': -296.0, 'status': 'ok'},
 'misc': {'tid': 10,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [10], 'y': [10]},
  'vals': {'x': [2.0], 'y': [15.0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2025, 1, 2, 2, 41, 4, 753000),
 'refresh_time': datetime.datetime(2025, 1, 2, 2, 41, 4, 753000)}

In [13]:
trial_val.results

[{'loss': -64.0, 'status': 'ok'},
 {'loss': -184.0, 'status': 'ok'},
 {'loss': 56.0, 'status': 'ok'},
 {'loss': -224.0, 'status': 'ok'},
 {'loss': 61.0, 'status': 'ok'},
 {'loss': -64.0, 'status': 'ok'},
 {'loss': -184.0, 'status': 'ok'},
 {'loss': 56.0, 'status': 'ok'},
 {'loss': -224.0, 'status': 'ok'},
 {'loss': 61.0, 'status': 'ok'},
 {'loss': -296.0, 'status': 'ok'},
 {'loss': -40.0, 'status': 'ok'},
 {'loss': 281.0, 'status': 'ok'},
 {'loss': 64.0, 'status': 'ok'},
 {'loss': 100.0, 'status': 'ok'},
 {'loss': 60.0, 'status': 'ok'},
 {'loss': -39.0, 'status': 'ok'},
 {'loss': 1.0, 'status': 'ok'},
 {'loss': -164.0, 'status': 'ok'},
 {'loss': 21.0, 'status': 'ok'}]

In [21]:
import pandas as pd

losses = [ loss_dict['loss'] for loss_dict in trial_val.results]
#losses

result_df = pd.DataFrame({'x': trial_val.vals['x'], 'y': trial_val.vals['y'], 'losses': losses})
result_df.head(20)


,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,-6.0,5.0,-64.0
6,-4.0,10.0,-184.0
7,4.0,-2.0,56.0
8,-4.0,12.0,-224.0
9,9.0,1.0,61.0


# XGBoost 하이퍼파라미터 최적화